In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import mysql.connector

# --- Extract Data from CSV ---
file_path = "/Users/shriyog/Downloads/Assignement 6/1.csv"  # Path to your new CSV file
data = pd.read_csv(file_path)

# --- Transform Data ---
# Rename columns (if necessary) and handle missing values
data.rename(columns={
    'Customer ID': 'customer_id',
    'First Name': 'first_name',
    'Last Name': 'last_name',
    'Email Address': 'email_address',
    'City': 'city',
    'State': 'state',
    'Country': 'country',
    'Total Spend': 'total_spend',
    'Last Purchase': 'last_purchase',
    'Notes': 'notes'
}, inplace=True)

# Fill missing values
data['first_name'].fillna('Unknown', inplace=True)
data['last_name'].fillna('Unknown', inplace=True)
data['email_address'].fillna('No Email', inplace=True)
data['city'].fillna('Unknown City', inplace=True)
data['state'].fillna('Unknown State', inplace=True)
data['country'].fillna('Unknown Country', inplace=True)
data['total_spend'].fillna(0, inplace=True)
data['last_purchase'].fillna('No Purchase', inplace=True)
data['notes'].fillna('No Notes', inplace=True)

# Convert data types if necessary
data['total_spend'] = pd.to_numeric(data['total_spend'], errors='coerce').fillna(0)

# --- Load Data into MySQL ---
try:
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="password",
        database="DW3"
    )
    cursor = connection.cursor()

    # Ensure table exists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS customer_data (
            customer_id VARCHAR(255),
            first_name VARCHAR(255),
            last_name VARCHAR(255),
            email_address VARCHAR(255),
            city VARCHAR(255),
            state VARCHAR(255),
            country VARCHAR(255),
            total_spend DECIMAL(10, 2),
            last_purchase VARCHAR(255),
            notes VARCHAR(255)
        );
    """)

    # Insert data into the table
    for _, row in data.iterrows():
        sql = """
        INSERT INTO customer_data (
            customer_id, first_name, last_name, email_address, city, state, 
            country, total_spend, last_purchase, notes
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (
            row['customer_id'], row['first_name'], row['last_name'], row['email_address'],
            row['city'], row['state'], row['country'], row['total_spend'], row['last_purchase'],
            row['notes']
        ))

    # Commit changes
    connection.commit()
    print("Data successfully loaded into MySQL!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


Data successfully loaded into MySQL!


/var/folders/b7/zsjw9l1579bg90by892m2zj00000gn/T/ipykernel_55941/420086220.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'No Purchase' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data['last_purchase'].fillna('No Purchase', inplace=True)
